In [ ]:
# install and load packages, you only need to do this once. 

rprofile_path = file.path(Sys.getenv("HOME"), ".Rprofile")
write('\noptions(repos=c(getOption(\'repos\'),
          CRAN=\'https://cloud.r-project.org\',
          USGS=\'https://owi.usgs.gov/R\'))\n',
      rprofile_path, 
      append =  TRUE)
install.packages('smwrBase')
install.packages('rloadest')
install.packages('car')
install.packages('unitted')
install.packages('dplyr')
packageurl <- "https://github.com/USGS-R/loadflex/archive/v1.0.1.tar.gz"
install.packages(packageurl, repos=NULL, type="source")
install.packages('dplyr')


library('loadflex')
library('rloadest')

In [ ]:

newnames <- as.vector(t(read.table(
  "RioPiedrasDaily.csv",
  sep=",",header=FALSE,nrows=1,skip=25,stringsAsFactors=FALSE)[1,]))

RioPiedrasDailyDischarge <- read.table(
  "RioPiedrasDaily.csv", #MPR1991-1996
  sep=",",header=FALSE,col.names=newnames,skip=27,stringsAsFactors=FALSE)
#MPR_SSForLoadest-1991.csv

RioPiedrasDailyDischarge$DateTime <- as.POSIXct(strptime(RioPiedrasDailyDischarge$datetime,format="%m/%d/%Y" ,tz="America/Puerto_Rico"))
RioPiedrasDailyDischarge$date<-  as.Date(RioPiedrasDailyDischarge$date, format("%m/%d/%Y"),tz="America/Puerto_Rico")
#RioPiedrasDischargeShort <- RioPiedrasDischarge[complete.cases(RioPiedrasDischarge["CFS"]),]


newnames <- as.vector(t(read.table(
  "RioPiedrasDischarge2.csv",
  sep=",",header=FALSE,nrows=1,skip=25,stringsAsFactors=FALSE)[1,]))

RioPiedrasDischarge <- read.table(
  "RioPiedrasDischarge2.csv", #MPR1991-1996
  sep=",",header=FALSE,col.names=newnames,skip=27,stringsAsFactors=FALSE)
#MPR_SSForLoadest-1991.csv
RioPiedrasDischarge$DateTime <- as.POSIXct(strptime(RioPiedrasDischarge$datetime,format="%m/%d/%Y %H:%M" ,tz="America/Puerto_Rico"))  #as.Date(RioPiedrasDischarge$datetime, format="%m/%d/%Y %H:%M")
#RioPiedrasDischarge$date<- as.POSIXct(strptime(RioPiedrasDischarge$datetime,format="%m/%d/%Y %H:%M" ,tz="America/Puerto_Rico"))
RioPiedrasDischarge$date <- as.Date(RioPiedrasDischarge$date, format("%m/%d/%Y"),tz="America/Puerto_Rico")

RioPiedrasDischargeWithDaily<- merge(RioPiedrasDischarge,RioPiedrasDailyDischarge, by=c('date'),all.x=TRUE)
#inspect the time period with a gap in the 15 minute record
RioPiedrasDischargeWithDailyShort <-RioPiedrasDischargeWithDaily[RioPiedrasDischargeWithDaily$DateTime.y >= as.POSIXct("2009-09-28") 
                                                                 & RioPiedrasDischargeWithDaily$DateTime.y <= as.POSIXct("2009-11-07"),]

#if 15 min record is NA fill it with daily record 

RioPiedrasDischargeWithDaily2 <- within(RioPiedrasDischargeWithDaily, CFS.x<- ifelse(is.na(CFS.x), CFS.y, CFS.x))
#inspect the time period with a gap in the 15 minute record
RioPiedrasDischargeWithDailyShort2 <-RioPiedrasDischargeWithDaily2[RioPiedrasDischargeWithDaily2$DateTime.y >= as.POSIXct("2009-09-28") 
                                                                   & RioPiedrasDischargeWithDaily2$DateTime.y <= as.POSIXct("2009-11-07"),]
RioPiedrasDischarge$CFS <- RioPiedrasDischargeWithDaily2$CFS.x

newnames <- as.vector(t(read.table(
  "RioPiedrasChem.csv",
  sep=",",header=FALSE,nrows=1,stringsAsFactors=FALSE)[1,]))

RP_chem <- read.table(
  "RioPiedrasChem.csv", 
  sep=",",header=FALSE,col.names=newnames,skip=1,stringsAsFactors=FALSE)


RP_chem$date <- as.POSIXct(strptime(RP_chem$Sample_Date  ,format="%m/%d/%Y",tz="America/Puerto_Rico"))
#RP_chem$date <- as.Date(RP_chem$date, format="%d/%m/%Y")
RioPiedrasDischarge$DateTime <- as.POSIXct(strptime(RioPiedrasDischarge$datetime,format="%m/%d/%Y %H:%M" ,tz="America/Puerto_Rico"))  #as.Date(RioPiedrasDischarge$datetime, format="%m/%d/%Y %H:%M")
RioPiedrasDischarge$date<- as.POSIXct(strptime(RioPiedrasDischarge$datetime,format="%m/%d/%Y %H:%M" ,tz="America/Puerto_Rico"))

RP_chemNa <- RP_chem[complete.cases(RP_chem["Na_mg_L"]),]
RP_chemNa <- RP_chemNa[complete.cases(RP_chemNa["CFS"]),]

#RioPiedrasDischarge <- RioPiedrasDischarge[c("DateTime","CFS")]

meta <- metadata(constituent="Na_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1")


lr <- loadReg2(loadReg(Na_mg_L ~ model(1), data=RP_chemNa,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

#create the rloadest report for Na
lrforreport <- loadReg(Na_mg_L ~ model(1), data=RP_chemNa,
        flow="CFS", dates="date", time.step="instantaneous", 
        flow.units="cfs", conc.units="mg/L", load.units="kg")
loadReport(lrforreport,"NaRioPiedras")

lc <- loadComp(reg.model=lr, interp.format="flux", 
                   interp.data=RP_chemNa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

#wis_reg <- loadReg2(loadReg(NO3_corrected_mgL ~ model(9), data=wisregdat,
#                            flow="Q_cms", dates="Date", time.step="instantaneous", 
#                            flow.units="cms", conc.units="mg/L", load.units="kg", station="Lamprey at Wiswall Dam, NH"))
#wis_cmp <- loadComp(reg.model=wis_reg, interp.format="conc", interp.data=wisintdat)


#wis_sensordata$Date <- as.POSIXct(strptime(wis_sensordata$Date, "%Y-%m-%d %H:%M:%S"))

RioPiedrasDischargeShort <- RioPiedrasDischarge[complete.cases(RioPiedrasDischarge["CFS"]),]
#RioPiedrasDischarge2012short <- RioPiedrasDischargeShort[RioPiedrasDischargeShort$DateTime >= as.POSIXct("2009-1-01") 
#                                               & RioPiedrasDischargeShort$DateTime <= as.POSIXct("2011-12-31"),]

preds_lrNa <- predictSolute(lr, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

preds_lrNaConc <- predictSolute(lr, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)
preds_lcNaConc <- predictSolute(lc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrNa <- aggregateSolute(preds_lrNa,meta, format="flux total", se.preds=preds_lrNa$se.pred, agg.by="calendar year")


preds_lcNa <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNa <- aggregateSolute(preds_lcNa,meta, format="flux total", se.preds=preds_lrNa$se.pred, agg.by="calendar year")


In [ ]:
#Ca
RP_chemCa <- RP_chem[complete.cases(RP_chem["Ca_mg_L"]),]
RP_chemCa <- RP_chemCa[complete.cases(RP_chemCa["CFS"]),]

meta <- metadata(constituent="Ca_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(Ca_mg_L ~ model(1), data=RP_chemCa,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemCa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrCa <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrCa <- aggregateSolute(preds_lrCa,meta, format="flux total", se.preds=preds_lrCa$se.pred, agg.by="calendar year")


#Mg
RP_chemMg <- RP_chem[complete.cases(RP_chem["Mg_mg_L"]),]
RP_chemMg <- RP_chemMg[complete.cases(RP_chemMg["CFS"]),]

meta <- metadata(constituent="Mg_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(Mg_mg_L ~ model(1), data=RP_chemMg,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemMg, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrMg <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrMg <- aggregateSolute(preds_lrMg,meta, format="flux total", se.preds=preds_lrMg$se.pred, agg.by="calendar year")



#DOC
RP_chemDOC <- RP_chem[complete.cases(RP_chem["DOC_mg_L"]),]
RP_chemDOC <- RP_chemDOC[complete.cases(RP_chemDOC["CFS"]),]

meta <- metadata(constituent="DOC_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(DOC_mg_L ~ model(1), data=RP_chemDOC,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemDOC, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcDOCConc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcDOCConc <- aggregateSolute(preds_lcDOCConc,meta, format="conc", se.preds=preds_lcDOCConc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemDOC, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrDOC <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrDOC <- aggregateSolute(preds_lrDOC,meta, format="flux total", se.preds=preds_lrDOC$se.pred, agg.by="calendar year")



#DIC
RP_chemDIC <- RP_chem[complete.cases(RP_chem["DIC_mg_L"]),]
RP_chemDIC <- RP_chemDIC[complete.cases(RP_chemDIC["CFS"]),]

meta <- metadata(constituent="DIC_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(DIC_mg_L ~ model(1), data=RP_chemDIC,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemDIC, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcDICConc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcDICConc <- aggregateSolute(preds_lcDICConc,meta, format="conc", se.preds=preds_lcDICConc$se.pred, agg.by="calendar year")


lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemDIC, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrDIC <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrDIC <- aggregateSolute(preds_lrDIC,meta, format="flux total", se.preds=preds_lrDIC$se.pred, agg.by="calendar year")




#TDN
RP_chemTDN <- RP_chem[complete.cases(RP_chem["TDN_mg_L"]),]
RP_chemTDN <- RP_chemTDN[complete.cases(RP_chemTDN["CFS"]),]

meta <- metadata(constituent="TDN_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(TDN_mg_L ~ model(1), data=RP_chemTDN,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemTDN, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcTDNConc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcTDNConc <- aggregateSolute(preds_lcTDNConc,meta, format="conc", se.preds=preds_lcTDNConc$se.pred, agg.by="calendar year")


lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemTDN, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrTDN <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrTDN <- aggregateSolute(preds_lrTDN,meta, format="flux total", se.preds=preds_lrTDN$se.pred, agg.by="calendar year")




#DON
RP_chemDON <- RP_chem[complete.cases(RP_chem["DON_mg_L"]),]
RP_chemDON <- RP_chemDON[complete.cases(RP_chemDON["CFS"]),]

meta <- metadata(constituent="DON_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(DON_mg_L ~ model(1), data=RP_chemDON,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemDON, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcDONConc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcDONConc <- aggregateSolute(preds_lcDONConc,meta, format="conc", se.preds=preds_lcDONConc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemDON, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrDON <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrDON <- aggregateSolute(preds_lrDON,meta, format="flux total", se.preds=preds_lrDON$se.pred, agg.by="calendar year")



#TSS
RP_chemTSS <- RP_chem[complete.cases(RP_chem["TSS_mg_L"]),]
RP_chemTSS <- RP_chemTSS[complete.cases(RP_chemTSS["CFS"]),]

meta <- metadata(constituent="TSS_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(TSS_mg_L ~ model(1), data=RP_chemTSS,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemTSS, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcTSSConc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcTSSConc <- aggregateSolute(preds_lcTSSConc,meta, format="conc", se.preds=preds_lcTSSConc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemTSS, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrTSS <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrTSS <- aggregateSolute(preds_lrTSS,meta, format="flux total", se.preds=preds_lrTSS$se.pred, agg.by="calendar year")



#NH4
RP_chemNH4 <- RP_chem[complete.cases(RP_chem["NH4.N_ug_L"]),]
RP_chemNH4 <- RP_chemNH4[complete.cases(RP_chemNH4["CFS"]),]

meta <- metadata(constituent="NH4.N_ug_L", flow="CFS", 
                 dates="date", conc.units="ug/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(NH4.N_ug_L ~ model(1), data=RP_chemNH4,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="ug/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemNH4, abs.or.rel.resids="relative", interp.function=triangularInterpolation)

preds_lcNH4Conc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNH4Conc <- aggregateSolute(preds_lcNH4Conc,meta, format="conc", se.preds=preds_lcNH4Conc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemNH4, abs.or.rel.resids="relative", interp.function=triangularInterpolation) #triangularInterpolation

preds_lrNH4 <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)
#aggs_lrNH4month <- aggregateSolute(preds_lrNH4,meta, format="flux total", se.preds=preds_lrNH4$se.pred, agg.by="month")

aggs_lrNH4 <- aggregateSolute(preds_lrNH4,meta, format="flux total", se.preds=preds_lrNH4$se.pred, agg.by="calendar year")


#Cl
RP_chemCl <- RP_chem[complete.cases(RP_chem["Cl_mg_L"]),]
RP_chemCl <- RP_chemCl[complete.cases(RP_chemCl["CFS"]),]

meta <- metadata(constituent="Cl_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(Cl_mg_L ~ model(1), data=RP_chemCl,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemCl, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcClConc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcClConc <- aggregateSolute(preds_lcClConc,meta, format="conc", se.preds=preds_lcClConc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemCl, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcCl <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcCl <- aggregateSolute(preds_lcCl,meta, format="flux total", se.preds=preds_lrCl$se.pred, agg.by="calendar year")


#NO3
RP_chemNO3 <- RP_chem[complete.cases(RP_chem["NO3.N_ugN_L."]),]
RP_chemNO3 <- RP_chemNO3[complete.cases(RP_chemNO3["CFS"]),]

meta <- metadata(constituent="NO3.N_ugN_L.", flow="CFS", 
                 dates="date", conc.units="ug/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(NO3.N_ugN_L. ~ model(1), data=RP_chemNO3,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="ug/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemNO3, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcNO3Conc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNO3Conc <- aggregateSolute(preds_lcNO3Conc,meta, format="conc", se.preds=preds_lcNO3Conc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemNO3, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrNO3 <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrNO3 <- aggregateSolute(preds_lrNO3,meta, format="flux total", se.preds=preds_lrNO3$se.pred, agg.by="calendar year")


#SO4
RP_chemSO4 <- RP_chem[complete.cases(RP_chem["SO4.S_mg_L"]),]
RP_chemSO4 <- RP_chemSO4[complete.cases(RP_chemSO4["CFS"]),]

meta <- metadata(constituent="SO4.S_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(SO4.S_mg_L ~ model(1), data=RP_chemSO4,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))


lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemSO4, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcSO4Conc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcSO4Conc <- aggregateSolute(preds_lcSO4Conc,meta, format="conc", se.preds=preds_lcSO4Conc$se.pred, agg.by="calendar year")


lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemSO4, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrSO4 <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrSO4 <- aggregateSolute(preds_lrSO4,meta, format="flux total", se.preds=preds_lrSO4$se.pred, agg.by="calendar year")




#F
RP_chemF <- RP_chem[complete.cases(RP_chem["F_mg_L"]),]
RP_chemF <- RP_chemF[complete.cases(RP_chemF["CFS"]),]

meta <- metadata(constituent="F_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(F_mg_L ~ model(1), data=RP_chemF,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))


lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemF, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcFConc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcFConc <- aggregateSolute(preds_lcFConc,meta, format="conc", se.preds=preds_lcFConc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemF, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrF <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrF <- aggregateSolute(preds_lrF,meta, format="flux total", se.preds=preds_lrF$se.pred, agg.by="calendar year")



#K
RP_chemK <- RP_chem[complete.cases(RP_chem["K_mg_L"]),]
RP_chemK <- RP_chemK[complete.cases(RP_chemK["CFS"]),]

meta <- metadata(constituent="K_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(K_mg_L ~ model(1), data=RP_chemK,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))



lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemK, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcKConc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcKConc <- aggregateSolute(preds_lcKConc,meta, format="conc", se.preds=preds_lcKConc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemK, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrK <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrK <- aggregateSolute(preds_lrK,meta, format="flux total", se.preds=preds_lrK$se.pred, agg.by="calendar year")



#PO4
RP_chemPO4 <- RP_chem[complete.cases(RP_chem["PO4.P_ug_P_L"]),]
RP_chemPO4 <- RP_chemPO4[complete.cases(RP_chemPO4["CFS"]),]

meta <- metadata(constituent="PO4.P_ug_P_L", flow="CFS", 
                 dates="date", conc.units="ug/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(PO4.P_ug_P_L ~ model(1), data=RP_chemPO4,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="ug/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=RP_chemPO4, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcPO4Conc <- predictSolute(lcConc, "conc",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcPO4Conc <- aggregateSolute(preds_lcPO4Conc,meta, format="conc", se.preds=preds_lcPO4Conc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=RP_chemPO4, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrPO4 <- predictSolute(lc, "flux",interval="prediction", RioPiedrasDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrPO4 <- aggregateSolute(preds_lrPO4,meta, format="flux total", se.preds=preds_lrPO4$se.pred, agg.by="calendar year")


